In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pwd

/content


In [2]:
#setting working directory
%cd /content/drive/MyDrive/master/data_internship

/content/drive/MyDrive/master/data_internship


In [3]:
!pwd

/content/drive/MyDrive/master/data_internship


In [17]:
%%capture
!pip install pyannote.audio
!pip install pyannote.core
!pip install torch
!pip install torchaudio
!pip install faster-whisper
!pip install --no-build-isolation nemo_toolkit[asr]==1.22.0
!pip install ffmpeg
#!pip install wget

In [5]:
#segment
from pyannote.audio import Audio
from pyannote.core import Segment
import torch, torchaudio
import ffmpeg
import os
#import librosa

#embed
from faster_whisper import WhisperModel
#from segment import segment_audio
#from embed import embed_audio
import tensorflow as tf

#diarize
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
import wget

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
[NeMo W 2024-03-05 14:15:44 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.


# Segmentation

In [6]:
audio = Audio(mono='downmix') #add parsing arg (for nemo we need mono)

def segmentation(trans_segment,
            waveform,
            #video,
            sr,
            seg_num,
            segment_audio=True,
            segment_video=False,
            save_into_file=False):
  '''
  Given a start and end timestamp from the transcription, it crops an audio file
  or segment returned as a torch tensor. Moreover, if save_into_file=True, it
  saves the segment into a .wav and .pt file. For default, turned off as it
  consumes storage memory.
  '''

  if segment_audio: #kathleen needs: .wav segs, .pt segs
    #segmenting
    start = trans_segment.start
    end = trans_segment.end
    clip = Segment(start, end)
    audio_segment, sr = audio.crop(waveform, clip) #as a tensor

    if save_into_file:
      #extracting file name and extension
      name, extension = os.path.splitext(waveform)
      if not os.path.exists("%s_segments" % (name)):
        os.makedirs("%s_segments" % (name))

      #saving segment into .wav
      torchaudio.save("./%s_segments/%s_seg_%x%s" % (name, name, seg_num, extension),
                          audio_segment, sr, format="wav")

      #saving segment into .pt
      torch.save(audio_segment, "./%s_segments/%s_seg_%x.pt" % (name, name, seg_num), _use_new_zipfile_serialization=False)

    return audio_segment

    #if segment_video: #kathleen needs: .mp4 segs
        #ffmpeg -i video -ss 00:01:00 -to 00:02:00 -c copy output_segmented_video.mp4

# Feature Extraction

In [8]:
def load_model():
  '''
  Loads feature extraction model, configuration and weights.
  '''

  # load the pre-trained checkpoints
  model_checkpoint_loc = 'WavLM-Large.pt'
  checkpoint = torch.load(model_checkpoint_loc) #should be .pt file
  cfg = WavLMConfig(checkpoint['cfg'])
  model = WavLM(cfg)
  model.load_state_dict(checkpoint['model'])
  model.eval()
  return model, cfg

def embed_audio(model, cfg, audio_segment, last_layer=True):
  '''
  Generates embeddings for the given audio file or segment. Extracts the
  representation of the last layer (last_layer=True), or the representation from
  each layer and performs a weighted sum (last_layer=False).
  '''

  if last_layer:
    if cfg.normalize:
        wav_input_16khz = torch.nn.functional.layer_norm(audio_segment, audio_segment.shape)
    rep = model.extract_features(wav_input_16khz)[0]

  if last_layer == False:
    if cfg.normalize:
      wav_input_16khz = torch.nn.functional.layer_norm(wav_input_16khz , wav_input_16khz.shape)
    rep, layer_results = model.extract_features(wav_input_16khz, output_layer=model.cfg.encoder_layers, ret_layer_results=True)[0]
    layer_reps = [x.transpose(0, 1) for x, _ in layer_results]

  return rep

In [38]:
#read from .py file to import functions
#import sys
#sys.path.append('/content/drive/MyDrive/master/data_internship')
#from WavLM import WavLM, WavLMConfig

In [12]:
from WavLM import WavLM, WavLMConfig
#loading feature extraction model
feat_model, cfg = load_model()

# Run

## Faster-whisper Transcription

In [13]:
model_size = "large-v3"

# Run on GPU with FP16
#model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
model = WhisperModel(model_size, device="cpu", compute_type="int8")

audioin = "testaudio.wav"

segments, info = model.transcribe(audioin, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

Detected language 'en' with probability 0.913472


## Segment and embed

In [ ]:
#--- PARAMS ---#
#sampling rate
sr = 16000
#segment number initialization
seg_num = 0

for segment in segments:
    #print(segment.text)
    #calling segmentation creates
    seg = segmentation(segment, audioin, sr, seg_num)
    seg_num+=1
    emb = embed_audio(feat_model, cfg, seg)
    print(emb)

#Diarization (in progress, don't run)

In [8]:
ROOT = os.getcwd()
temp_path = os.path.join(ROOT, "temp_outputs")

In [14]:
def create_config(output_dir):
    DOMAIN_TYPE = "general" #
    CONFIG_FILE_NAME = f"diar_infer_{DOMAIN_TYPE}.yaml"
    CONFIG_URL = f"https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/{CONFIG_FILE_NAME}"
    MODEL_CONFIG = os.path.join(output_dir, CONFIG_FILE_NAME)
    if not os.path.exists(MODEL_CONFIG):
        MODEL_CONFIG = wget.download(CONFIG_URL, output_dir)

In [ ]:
msdd_model = NeuralDiarizer(cfg=create_config(temp_path)).to("cuda")
msdd_model.diarize()

del msdd_model
torch.cuda.empty_cache()